In [1]:
import pandas as pd
import numpy as np
import json
from keras.layers import Dense, LSTM
from keras.models import Sequential, load_model

/Users/giovanni/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
features_name = ['AcX', 'AcY', 'AcZ', 'Pres', 'GyX', 'GyY', 'GyZ']

In [3]:
with open('lines.json', 'r') as f:
    data_lines = json.load(f)

with open('lines_2.json', 'r') as f:
    data_lines += json.load(f)

for elem in range(len(data_lines)):
    for arr in range(len(data_lines[elem])):
        tmp = []
        for f in features_name:
            tmp.append(data_lines[elem][arr][f])
        data_lines[elem][arr] = tmp

In [4]:
with open('circles.json', 'r') as f:
    data_circles = json.load(f)

for elem in range(len(data_circles)):
    for arr in range(len(data_circles[elem])):
        tmp = []
        for f in features_name:
            tmp.append(data_circles[elem][arr][f])
        data_circles[elem][arr] = tmp
        

In [5]:
with open('lines.json', 'r') as f:
    data_lines = json.load(f)

with open('lines_2.json', 'r') as f:
    data_lines += json.load(f)
    
def new_scale_df(data_frame, gyro, accl, pres):
    
    def scale(value, max_, min_):
        value -= min_
        value /= max_ - min_
    
    copy = data_frame.copy() 
    scale(copy['AcX'], accl, -accl)
    scale(copy['AcY'], accl, -accl)
    scale(copy['AcZ'], accl, -accl)
    scale(copy['GyX'], gyro, -gyro)
    scale(copy['GyY'], gyro, -gyro)
    scale(copy['GyZ'], gyro, -gyro)
    scale(copy['Pres'], pres, 0)

    return copy
    
def generator(features_name, data, labels):
    assert len(data) == len(labels)
    while True:
        for elem in range(len(data)):
            word_array = []
            for arr in range(len(data[elem])):
                cp_data = new_scale_df(data[elem][arr], 250, 2, 1024)
                tmp = []
                for f in features_name:
                    tmp.append(cp_data[f])
                word_array.append(tmp)
            yield np.reshape(np.array(word_array), (1, len(word_array), 7)), [labels[elem]]

g_lines = generator(features_name, data_lines, [1] * len(data_lines))

In [6]:
with open('circles.json', 'r') as f:
    data_circles = json.load(f)
    
g_circles = generator(features_name, data_circles, [0] * len(data_circles))

In [7]:
with open('dario.json', 'r') as f:
    data_d = json.load(f)
g_dario = generator(features_name, data_d[:-3], [1] * 7)
g_dario_test = generator(features_name, data_d[-3:], [1] * 3)

In [8]:
with open('antonio.json', 'r') as f:
    data_a = json.load(f)
    
g_antonio = generator(features_name, data_a[:-3], [0] * 8)
g_antonio_test = generator(features_name, data_a[-3:], [0] * 3)

In [9]:
with open('taraz.json', 'r') as f:
    data_t = json.load(f)
    
g_taraz = generator(features_name, data_t[:-3], [0] * 7)
g_taraz_test = generator(features_name, data_t[-3:], [0] * 3)

In [10]:
with open('giovanni.json', 'r') as f:
    data_g = json.load(f)
    
g_giovanni = generator(features_name, data_g[:-3], [0] * 5)
g_giovanni_test = generator(features_name, data_g[-3:], [0] * 3)

In [11]:
def mix_generator(g1, g2):
    while True:
        yield next(g1)
        yield next(g2)

g_mix = mix_generator(g_lines, g_circles)
g_dario_giovanni = mix_generator(g_dario, g_giovanni)
g_dario_antonio = mix_generator(g_dario, g_antonio)
g_dario_taraz = mix_generator(g_dario, g_taraz)
g_dario_giovanni_antonio = mix_generator(g_dario_giovanni, g_dario_antonio)
g_dario_tutti = mix_generator(g_dario_giovanni_antonio, g_dario_taraz)

In [12]:
def generator_one_against_all(path_genuine, paths_impostors):
    with open(path_genuine, 'r') as f:
        genuine_data = json.load(f)
        
    train_ratio = round(len(genuine_data) / 5)
    assert len(genuine_data[:-train_ratio]) + len(genuine_data[-train_ratio:]) == len(genuine_data)
    genuine_generator = generator(features_name, genuine_data[:-train_ratio], [1] * (len(genuine_data) - train_ratio))
    genuine_generator_test = generator(features_name, genuine_data[-train_ratio:], [1] * train_ratio)
    
    generator_all = genuine_generator
    generator_all_test = genuine_generator_test
    
    for path in paths_impostors:
        with open(path, 'r') as f:
            impostor_data = json.load(f)
        train_ratio = round(len(impostor_data) / 5)
        assert len(impostor_data[:-train_ratio]) + len(impostor_data[-train_ratio:]) == len(impostor_data)
        generator_all = mix_generator(generator_all, generator(features_name, impostor_data[:-train_ratio], [0] * (len(impostor_data) - train_ratio)))
        generator_all_test = mix_generator(generator_all_test, generator(features_name, impostor_data[-train_ratio:], [0] * train_ratio))
        
    return generator_all, generator_all_test

impostors = ['giovanni.json', 'taraz.json']
g_dario_impostors, g_dario_impostors_test = generator_one_against_all('dario.json', impostors)

In [13]:
model_dario = Sequential()
model_dario.add(LSTM(batch_size=1, input_shape=(None, 7), units=200, activation="sigmoid", return_sequences=False, recurrent_activation="hard_sigmoid"))
model_dario.add(Dense(1, activation='sigmoid'))

In [14]:
model_dario.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
hist = model_dario.fit_generator(g_dario_impostors, epochs=30, steps_per_epoch=42)

Epoch 1/30
42/42 [==============================] - 7s 178ms/step - loss: 0.6387 - acc: 0.6429
Epoch 2/30
42/42 [==============================] - 6s 147ms/step - loss: 0.5123 - acc: 0.7619
Epoch 3/30
42/42 [==============================] - 6s 145ms/step - loss: 0.4916 - acc: 0.7381
Epoch 4/30
42/42 [==============================] - 6s 148ms/step - loss: 0.4558 - acc: 0.7619
Epoch 5/30
42/42 [==============================] - 6s 146ms/step - loss: 0.4475 - acc: 0.7381
Epoch 6/30
42/42 [==============================] - 6s 150ms/step - loss: 0.3984 - acc: 0.7619
Epoch 7/30
42/42 [==============================] - 6s 144ms/step - loss: 0.4245 - acc: 0.7619
Epoch 8/30
42/42 [==============================] - 6s 147ms/step - loss: 0.3615 - acc: 0.8810
Epoch 9/30
42/42 [==============================] - 6s 149ms/step - loss: 0.3711 - acc: 0.8571
Epoch 10/30
42/42 [==============================] - 6s 147ms/step - loss: 0.3603 - acc: 0.8571
Epoch 11/30
42/42 [==============================

In [16]:
model_dario.evaluate_generator(g_dario_impostors_test, verbose=1, steps=6)

6/6 [==============================] - 0s 41ms/step


[0.24814651627093554, 0.8333333333333334]

In [17]:
out_dario = model_dario.evaluate_generator(g_dario_test, verbose=1, steps=3)
out_giovanni = model_dario.evaluate_generator(g_giovanni_test, verbose=1, steps=3)
out_taraz = model_dario.evaluate_generator(g_taraz_test, verbose=1, steps=3)

out_dario, out_giovanni, out_taraz

3/3 [==============================] - 0s 29ms/step


([0.47930579880873364, 0.6666666666666666],
 [0.10888240362207095, 1.0],
 [0.04201492046316465, 1.0])

In [18]:
model_dario.save("model_dario.h5")
del model_dario

In [19]:
model_dario_reloaded = load_model("model_dario.h5")

In [20]:
out_dario_reloaded = model_dario_reloaded.evaluate_generator(g_dario_test, verbose=1, steps=3)

out_dario_reloaded

3/3 [==============================] - 0s 59ms/step


[0.47930579880873364, 0.6666666666666666]

In [21]:
# QUA SOTTO C'È ROBA VECCHIA

with open('lines.json', 'r') as f:
    data = json.load(f)
    
X_train = []
y_train = []
for i in range(len(data)):
    dfl = pd.DataFrame(data = data[i])
    for j in range(len(dfl)):
        X_train.append(dfl.iloc[j].values)
        y_train.append(0)           # lines
 

with open('circles.json', 'r') as f:
    data = json.load(f)

dfc = pd.DataFrame(data = data)    
for i in range(len(data)):
    dfc = pd.DataFrame(data = data[i])
    for j in range(len(dfc)):
        X_train.append(dfc.iloc[j].values)
        y_train.append(1)           # circles
    
    
with open('mix.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data = data)    
mix = []
for i in range(len(df)):
    mix.append(pd.DataFrame(data = data[i]))


X_train = pd.DataFrame(X_train).values
y_train1 = np.array(y_train[:158])
X_train = np.reshape(X_train, (1, X_train.shape[0], 7))

X_train1 = np.reshape(X_train1, (1, -1, 7))
y_train1 = np.reshape(y_train1, (1, y_train1.shape[0], 1))
print(X_train.shape)
print(y_train)

model = Sequential()
model.add(LSTM(batch_size=1, input_shape=(None, 7), units=200, activation="sigmoid", return_sequences=True, recurrent_activation="hard_sigmoid"))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print(X_train1.shape)

hist = model.fit(X_train1, y_train1, batch_size=1, epochs=10, verbose=1)